# Political Bias Classification with BERT Variants

This Jupyter notebook walks through training four BERT-based classifiers on a news-bias dataset. It is GPU-ready and uses PyTorch & Hugging Face Transformers.

## 1. Install and Import Dependencies

In [ ]:
!pip install transformers datasets torch scikit-learn tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
import pandas as pd
from tqdm.auto import tqdm
from torch.cuda.amp import autocast, GradScaler
from torch.optim import AdamW
import pickle

# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


## 2. Load and Prepare the Data

In [4]:
# Path to your CSV
csv_path = "/content/political.csv"

df = pd.read_csv(csv_path)[:2500]

In [3]:
print(df.label.value_counts())

label
0    7723
2    5554
1    4008
Name: count, dtype: int64


In [5]:
# Train/test split
train_df, val_df = train_test_split(df, test_size=0.1, stratify=df.label, random_state=42)

## 3. Create a PyTorch Dataset

In [9]:
class NewsBiasDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = int(self.labels[idx])
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            padding='max_length',
            return_tensors='pt'
        )
        return {
            'input_ids': encoding.input_ids.squeeze(),
            'attention_mask': encoding.attention_mask.squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

## 4. Define Training and Evaluation Loops

In [10]:
class BertANNClassifier(nn.Module):
    def __init__(self, model_name, num_labels, d1=256, d2=128, dropout=0.3):
        super().__init__()
        self.bert = BertModel.from_pretrained(model_name)
        # enable gradient checkpointing to reduce memory
        self.bert.gradient_checkpointing_enable()
        hidden_size = self.bert.config.hidden_size
        self.fc1 = nn.Linear(hidden_size, d1)
        self.fc2 = nn.Linear(d1, d2)
        self.out = nn.Linear(d2, num_labels)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        cls = outputs.last_hidden_state[:, 0, :]
        x = self.dropout(cls)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        return self.out(x)

In [11]:
def train_epoch(model, loader, optimizer, scheduler, criterion, scaler):
    model.train()
    losses, preds, true = [], [], []
    for batch in tqdm(loader, desc='Training'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        with autocast():
            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        losses.append(loss.item())
        preds.extend(torch.argmax(logits, dim=1).cpu().numpy())
        true.extend(labels.cpu().numpy())

    return sum(losses)/len(losses), accuracy_score(true, preds), f1_score(true, preds, average='weighted')


def eval_epoch(model, loader, criterion):
    model.eval()
    losses, preds, true = [], [], []
    with torch.no_grad():
        for batch in tqdm(loader, desc='Evaluating'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)

            losses.append(loss.item())
            preds.extend(torch.argmax(logits, dim=1).cpu().numpy())
            true.extend(labels.cpu().numpy())
    return sum(losses)/len(losses), accuracy_score(true, preds), f1_score(true, preds, average='weighted')

## 5. Train BERT Variants

In [12]:
model_names = [
    'bert-base-uncased',
    'distilbert-base-uncased',
    'bert-base-cased'
]
num_labels = df.label.nunique()
results = {}

# Helper to create DataLoader
def make_loader(ds, batch_size):
     return DataLoader(
         ds, batch_size=batch_size,
         shuffle=(batch_size>1), num_workers=4, pin_memory=True
     )

for name in model_names:
    print(f"=== Training {name} + ANN ===")
    tokenizer = BertTokenizer.from_pretrained(name)
    train_ds = NewsBiasDataset(train_df.text.values, train_df.label.values, tokenizer)
    val_ds   = NewsBiasDataset(val_df.text.values,   val_df.label.values,   tokenizer)
    train_loader = make_loader(train_ds, batch_size=8)
    val_loader   = make_loader(val_ds,   batch_size=16)

    model = BertANNClassifier(name, num_labels).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=2e-5)
    total_steps = len(train_loader) * 10  # allow up to 10 epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=total_steps
    )
    scaler = GradScaler()

    best_f1 = 0.0
    patience, patience_counter = 2, 0  # early stopping after 2 epochs without improvement
    max_epochs = 10
    for epoch in range(1, max_epochs + 1):
        t_loss, t_acc, t_f1 = train_epoch(model, train_loader, optimizer, scheduler, criterion, scaler)
        v_loss, v_acc, v_f1 = eval_epoch(model, val_loader, criterion)
        print(f"Epoch {epoch}: Train F1={t_f1:.4f}  Val F1={v_f1:.4f}")

        # Early stopping logic
        if v_f1 > best_f1:
            best_f1 = v_f1
            patience_counter = 0
            # Save best checkpoints
            model.bert.save_pretrained(f"./ckpt/{name}_bert_best")
            torch.save(model.state_dict(), f"./ckpt/{name}_ann_best.pt")
            # Also save full model object as a pickle
            with open(f"./ckpt/{name}_full_model.pkl", "wb") as f:
                pickle.dump(model, f)
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Stopping early at epoch {epoch} (no improvement in {patience} epochs)")
                break

    results[name] = best_f1



=== Training bert-base-uncased + ANN ===


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Training:   0%|          | 0/282 [00:00<?, ?it/s]

<ipython-input-11-0cc8821446aa>:10: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Evaluating:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 5: Train F1=0.9458  Val F1=0.7113
Stopping early at epoch 5 (no improvement in 2 epochs)


In [13]:
print("Best validation F1 scores:")

for name, f1 in results.items():
    print(f"{name}: {f1:.4f}")

Best validation F1 scores:
bert-base-uncased: 0.7075
distilbert-base-uncased: 0.2772
bert-base-cased: 0.7610
